<a href="https://colab.research.google.com/github/kenwkliu/ideas/blob/master/colab/Word2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

from string import punctuation

import nltk
from nltk.corpus import gutenberg
from nltk.data import find
nltk.download('gutenberg')
nltk.download('punkt')
nltk.download('word2vec_sample')

import gensim
from gensim.models import word2vec

In [0]:
gutenberg.fileids()

In [0]:
# get the KJV bible
bible_kjv_sents = gutenberg.sents('bible-kjv.txt') 

[print(bible_kjv_sents[i]) for i in range(10)]
print("\n No. of sentences: ", len(bible_kjv_sents))

In [0]:
# preprocess the dataset
preprocessed = [[word.lower() for word in sent if word not in punctuation] for sent in bible_kjv_sents]

[print(preprocessed[i]) for i in range(10)]
print("\n No. of sentences: ", len(preprocessed))

In [0]:
model = word2vec.Word2Vec(preprocessed, min_count=5, size=64, window=5)

print("No.of words:", len(model.wv.vocab))
print("Model dimensions:", model.wv.vector_size)

In [0]:
model.wv.most_similar(["god"])

In [0]:
model.wv.most_similar(["jesus"])

In [0]:
model.wv.most_similar(["moses"])

In [0]:
model.wv.most_similar(["judas"])

In [0]:
# read the pre-trained Google News word2vec
word2vec_sample = str(find('models/word2vec_sample/pruned.word2vec.txt'))
model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_sample, binary=False)

print("No.of words:", len(model.vocab))

#Each word is represented in the space of 300 dimensions:
print("Model dimensions:", model.vector_size)

In [0]:
model.wv.most_similar(positive="university", topn=10)

In [0]:
model.doesnt_match('breakfast cereal dinner lunch'.split())

In [0]:
model.most_similar(positive=['woman','king'], negative=['man'], topn = 1)

In [0]:
model.most_similar(positive=['Paris','Germany'], negative=['Berlin'], topn = 1)

In [0]:
# show the word vector
model.wv['woman']

In [0]:
model.wv['man']